# Week 7 Assignment: Part 3 - Practical Audit

## Task: Audit a Dataset for Bias

**Dataset:** COMPAS Recidivism Dataset.

**Goal:**
1. Use Python and `aif360` to analyze racial bias in risk scores.
2. Generate visualizations.
3. Write a 300-word report summarizing findings and remediation steps.

In [ ]:
# Install necessary libraries
import sys
!{sys.executable} -m pip install aif360 pandas matplotlib numpy requests

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import io
import os

from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import Reweighing

## 1. Load Data

We will download the COMPAS dataset directly and load it into a pandas DataFrame. Then we will convert it into an `aif360` `BinaryLabelDataset`.

In [ ]:
# Download the COMPAS dataset
url = "https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv"
response = requests.get(url)
content = response.content
df = pd.read_csv(io.BytesIO(content))

# Preprocessing similar to ProPublica's analysis
df = df[
    (df["days_b_screening_arrest"] <= 30) &
    (df["days_b_screening_arrest"] >= -30) &
    (df["is_recid"] != -1) &
    (df["c_charge_degree"] != "O") &
    (df["score_text"] != "N/A")
]

# Select relevant columns
# race: Protected attribute
# two_year_recid: Target variable (1 = recidivated, 0 = did not)
# But usually for fairness we look at the *prediction* bias. 
# Here we are auditing the *dataset* itself (historical bias) or the *risk scores* if we treat them as predictions.
# Let's analyze the bias in the ground truth (recidivism) vs race, 
# OR more commonly, we analyze the bias in the 'decile_score' or 'score_text' acting as a proxy for prediction.
# For this audit, let's treat 'two_year_recid' as the label we want to predict, and check if the dataset itself has disparate impact.
# OR we can treat 'score_text' (Low/Medium/High) as the prediction.
# Let's create a binary prediction from score_text: High/Medium = 1 (High Risk), Low = 0 (Low Risk)

df['high_risk'] = df['score_text'].apply(lambda x: 1 if x in ['High', 'Medium'] else 0)

# We will analyze the bias in these 'high_risk' assignments.

dataset_df = df[['race', 'high_risk']]

# Filter for African-American and Caucasian for direct comparison
dataset_df = dataset_df[dataset_df['race'].isin(['African-American', 'Caucasian'])]

# Map race to binary: African-American = 0 (Unprivileged), Caucasian = 1 (Privileged)
# Note: This mapping is arbitrary for the tool, but we must define who is 'privileged' in the context of the metric.
# Usually, 'privileged' means the group that historically has the advantage (e.g., lower risk scores).
dataset_df['race_binary'] = dataset_df['race'].apply(lambda x: 1 if x == 'Caucasian' else 0)

# Create BinaryLabelDataset
dataset = BinaryLabelDataset(
    favorable_label=0, # 0 means Low Risk (Good outcome)
    unfavorable_label=1, # 1 means High Risk (Bad outcome)
    df=dataset_df,
    label_names=['high_risk'],
    protected_attribute_names=['race_binary']
)

privileged_groups = [{'race_binary': 1}] # Caucasian
unprivileged_groups = [{'race_binary': 0}] # African-American

print("Dataset loaded successfully.")
print(f"Total records: {len(dataset_df)}")
print(dataset_df['race'].value_counts())

## 2. Analyze Racial Bias

We will calculate the **Disparate Impact** and **Mean Difference** (Statistical Parity Difference).

*   **Disparate Impact:** Ratio of the rate of favorable outcome for the unprivileged group to the privileged group. Ideal is 1.0.
*   **Mean Difference:** Difference in the rate of favorable outcomes. Ideal is 0.0.

In [ ]:
metric = BinaryLabelDatasetMetric(
    dataset,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)

disparate_impact = metric.disparate_impact()
mean_difference = metric.mean_difference()

print(f"Disparate Impact: {disparate_impact:.4f}")
print(f"Mean Difference: {mean_difference:.4f}")

if disparate_impact < 0.8:
    print("\nWarning: Disparate impact is below 0.8, indicating significant bias against the unprivileged group.")

## 3. Visualizations

Let's visualize the proportion of individuals labeled as "High Risk" (Unfavorable) vs "Low Risk" (Favorable) for each group.

In [ ]:
# Calculate rates
priv_indices = (dataset_df['race_binary'] == 1)
unpriv_indices = (dataset_df['race_binary'] == 0)

priv_high_risk_rate = dataset_df[priv_indices]['high_risk'].mean()
unpriv_high_risk_rate = dataset_df[unpriv_indices]['high_risk'].mean()

print(f"High Risk Rate (Caucasian): {priv_high_risk_rate:.2%}")
print(f"High Risk Rate (African-American): {unpriv_high_risk_rate:.2%}")

# Plot
groups = ['Caucasian', 'African-American']
rates = [priv_high_risk_rate, unpriv_high_risk_rate]

plt.figure(figsize=(8, 6))
plt.bar(groups, rates, color=['blue', 'red'], alpha=0.7)
plt.ylabel('Proportion Labeled High/Medium Risk')
plt.title('Racial Bias in COMPAS Risk Scores')
plt.ylim(0, 1)
for i, v in enumerate(rates):
    plt.text(i, v + 0.02, f"{v:.1%}", ha='center', fontweight='bold')
plt.show()

## 4. Audit Report

### Findings

The analysis of the COMPAS dataset reveals significant racial bias in the risk scores assigned to defendants. Specifically, we analyzed the 'score_text' variable, categorizing 'High' and 'Medium' scores as high-risk (unfavorable outcome) and 'Low' scores as low-risk (favorable outcome).

The **Disparate Impact** metric is significantly below 1.0 (typically around 0.6-0.7 in this dataset), indicating that African-American defendants are far less likely to receive a 'Low Risk' score compared to Caucasian defendants. Conversely, the visualization shows that African-American defendants are labeled as 'High/Medium Risk' at a much higher rate than their Caucasian counterparts.

This confirms that the COMPAS algorithm, or the data it relies on, disproportionately flags African-American defendants as potential recidivists. This disparity exists even when controlling for prior crimes in more detailed analyses, suggesting systemic bias in the underlying data or the model's weighting of features.

### Remediation Steps

To address this bias, several steps should be taken:

1.  **Pre-processing (Reweighing):** We can use algorithms like `Reweighing` from the `aif360` toolkit to adjust the weights of the training examples. This would give more weight to favorable examples from the unprivileged group and less to unfavorable ones, helping to neutralize the bias in the training data before a model is built.
2.  **In-processing (Adversarial Debiasing):** If training a new model, we could use adversarial debiasing techniques where the model tries to predict recidivism while simultaneously minimizing its ability to predict the protected attribute (race).
3.  **Post-processing (Calibrated EqOdds):** We could adjust the decision thresholds for different groups to ensure that the False Positive Rates are equal across groups (Equal Opportunity), ensuring that innocent individuals from both groups are treated similarly.
4.  **Transparency and Human Oversight:** The risk scores should not be used as the sole determinant for sentencing or bail. Judges must be educated about these biases and use the scores only as one of many factors.